
<br>
======================================================================<br>
A demo of structured Ward hierarchical clustering on an image of coins<br>
======================================================================<br>
Compute the segmentation of a 2D image with Ward hierarchical<br>
clustering. The clustering is spatially constrained in order<br>
for each segmented region to be in one piece.<br>


Author : Vincent Michel, 2010<br>
         Alexandre Gramfort, 2011<br>
License: BSD 3 clause

In [ ]:
print(__doc__)

In [ ]:
import time as time

In [ ]:
import numpy as np
from distutils.version import LooseVersion
from scipy.ndimage.filters import gaussian_filter

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import skimage
from skimage.data import coins
from skimage.transform import rescale

In [ ]:
from sklearn.feature_extraction.image import grid_to_graph
from sklearn.cluster import AgglomerativeClustering

these were introduced in skimage-0.14

In [ ]:
if LooseVersion(skimage.__version__) >= '0.14':
    rescale_params = {'anti_aliasing': False, 'multichannel': False}
else:
    rescale_params = {}

#############################################################################<br>
Generate data

In [ ]:
orig_coins = coins()

Resize it to 20% of the original size to speed up the processing<br>
Applying a Gaussian filter for smoothing prior to down-scaling<br>
reduces aliasing artifacts.

In [ ]:
smoothened_coins = gaussian_filter(orig_coins, sigma=2)
rescaled_coins = rescale(smoothened_coins, 0.2, mode="reflect",
                         **rescale_params)

In [ ]:
X = np.reshape(rescaled_coins, (-1, 1))

#############################################################################<br>
Define the structure A of the data. Pixels connected to their neighbors.

In [ ]:
connectivity = grid_to_graph(*rescaled_coins.shape)

#############################################################################<br>
Compute clustering

In [ ]:
print("Compute structured hierarchical clustering...")
st = time.time()
n_clusters = 27  # number of regions
ward = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward',
                               connectivity=connectivity)
ward.fit(X)
label = np.reshape(ward.labels_, rescaled_coins.shape)
print("Elapsed time: ", time.time() - st)
print("Number of pixels: ", label.size)
print("Number of clusters: ", np.unique(label).size)

#############################################################################<br>
Plot the results on an image

In [ ]:
plt.figure(figsize=(5, 5))
plt.imshow(rescaled_coins, cmap=plt.cm.gray)
for l in range(n_clusters):
    plt.contour(label == l,
                colors=[plt.cm.nipy_spectral(l / float(n_clusters)), ])
plt.xticks(())
plt.yticks(())
plt.show()